In [1]:
from catboost import Pool, CatBoostClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df = df.drop(['customerID'], axis=1)

In [4]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
le = preprocessing.LabelEncoder()

cols = []
for col in df.columns:
    if df[col].dtype == 'object':
        cols.append(col)
df[cols] = df[cols].apply(le.fit_transform)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,2505,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1466,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,157,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1400,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,925,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   int32  
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   int32  
 3   Dependents        7043 non-null   int32  
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   int32  
 6   MultipleLines     7043 non-null   int32  
 7   InternetService   7043 non-null   int32  
 8   OnlineSecurity    7043 non-null   int32  
 9   OnlineBackup      7043 non-null   int32  
 10  DeviceProtection  7043 non-null   int32  
 11  TechSupport       7043 non-null   int32  
 12  StreamingTV       7043 non-null   int32  
 13  StreamingMovies   7043 non-null   int32  
 14  Contract          7043 non-null   int32  
 15  PaperlessBilling  7043 non-null   int32  
 16  PaymentMethod     7043 non-null   int32  


In [7]:
# Используем StandardScaler для нормализации данных.
scaler = StandardScaler()

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Churn'], axis=1), 
                                                    df['Churn'], test_size=0.30, 
                                                    random_state=42, stratify=df['Churn'])

score_all_models = {}

ValueError: Expected 2D array, got 1D array instead:
array=[ 5.  3.  3. ... 71. 24.  6.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [9]:
X_train

array([[-1.00528781, -0.43814715, -0.96295739, ...,  0.40179108,
         0.50428615, -0.00908875],
       [-1.00528781,  2.28233828, -0.96295739, ...,  0.40179108,
         0.72418915, -0.77874337],
       [-1.00528781, -0.43814715,  1.03846754, ..., -0.53432897,
         0.33729214, -0.80728825],
       ...,
       [-1.00528781, -0.43814715,  1.03846754, ...,  0.40179108,
         1.46491504,  1.35789397],
       [-1.00528781, -0.43814715,  1.03846754, ..., -0.53432897,
        -1.47318743,  0.40639788],
       [ 0.99474   , -0.43814715, -0.96295739, ...,  1.33791114,
        -1.4649204 , -1.65359116]])

# Testing Default Params

In [10]:
lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)
score = f1_score(y_test, y_pred)
print(score)
score_all_models['LGBM_default'] = score

0.11556982343499199


In [11]:
cbr = CatBoostClassifier(logging_level='Silent')
cbr.fit(X_train, y_train)
y_pred = cbr.predict(X_test)
score = f1_score(y_test, y_pred)
print(score)
score_all_models['CatBoost_default'] = score

0.0


In [12]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
score = f1_score(y_test, y_pred)
print(score)
score_all_models['XGB_default'] = score

[13:42:06] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.19287833827893175


In [13]:
skl = GradientBoostingClassifier().fit(X_train, y_train)
y_pred = skl.predict(X_test)
score = f1_score(y_test, y_pred)
print(score)
score_all_models['GradBoostSklearn_default'] = score

0.3917098445595855


# Testing GridSearchCV

In [14]:
from sklearn.model_selection import GridSearchCV

def estimate_grid_search(model, grid, X=X_train, y=y_train, name=None):
    clf = GridSearchCV(model, grid, verbose = 1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.best_params_)
    score = f1_score(y_test, y_pred)
    print(score)
    score_all_models[name] = score

In [15]:
grid = {'learning_rate': [0.05, 0.07, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [3, 4, 5, 7, 10],
        'iterations': [100, 200, 300]}
cbc = CatBoostClassifier(logging_level='Silent')
estimate_grid_search(cbc, grid, X=X_train, y=y_train, name='CatBoost_tuned')

Fitting 5 folds for each of 180 candidates, totalling 900 fits
{'depth': 10, 'iterations': 100, 'l2_leaf_reg': 7, 'learning_rate': 0.1}
0.0


In [16]:
grid = {'learning_rate': [0.03, 0.1],
        'max_depth': [-1, 4, 6, 8],
        'reg_lambda': [1, 3, 5, 7, 9]}
lgbm = lgb.LGBMClassifier()
estimate_grid_search(lgbm, grid, X=X_train, y=y_train, name='LGBM_tuned')

Fitting 5 folds for each of 40 candidates, totalling 200 fits
{'learning_rate': 0.03, 'max_depth': 4, 'reg_lambda': 1}
0.3132530120481928


In [17]:
grid = {'learning_rate': [0.05, 0.07, 0.1],
        'n_estimators': [100, 200, 500],
        'max_depth': [1, 4, 6, 8],
        'subsample':[0.2, 0.5, 0.7]}
skl = GradientBoostingClassifier()
estimate_grid_search(skl, grid, X=X_train, y=y_train, name='GradBoostSklearn_tuned')

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'learning_rate': 0.07, 'max_depth': 1, 'n_estimators': 200, 'subsample': 0.5}
0.0


In [18]:
grid = {'learning_rate': [0.05, 0.07, 0.1],
        'n_estimators': [100, 200, 300],
        'max_depth': [1, 4, 6, 8],
        'subsample':[0.2, 0.5, 0.7]}
xgb = XGBClassifier(verbosity=0, use_label_encoder=False)
estimate_grid_search(xgb, grid, X=X_train, y=y_train, name='XGB_tuned')

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 200, 'subsample': 0.7}
0.0


In [19]:
score_all_models

{'LGBM_default': 0.11556982343499199,
 'CatBoost_default': 0.0,
 'XGB_default': 0.19287833827893175,
 'GradBoostSklearn_default': 0.3917098445595855,
 'CatBoost_tuned': 0.0,
 'LGBM_tuned': 0.3132530120481928,
 'GradBoostSklearn_tuned': 0.0,
 'XGB_tuned': 0.0}

In [20]:
pd.DataFrame([score_all_models])

,LGBM_default,CatBoost_default,XGB_default,GradBoostSklearn_default,CatBoost_tuned,LGBM_tuned,GradBoostSklearn_tuned,XGB_tuned
0,0.11557,0.0,0.192878,0.39171,0.0,0.313253,0.0,0.0
